# 求$\sqrt{n}$

## 方法1：二分法

在一个区$(left, right)$内寻找$\sqrt{n}$，我们认为答案就是区间的中点。为了满足精度要求，我们一直用二分法缩小区间，直到区间大小小于规定的精度时，返回区间的中点作为答案。

In [ ]:
def sqrt1(n: float, precision=1e-10) -> float:
    # 如果n < 1，则在区间(0,1)中寻找；如果n >= 1，则在区间(1, n)中寻找
    left, right = (0, 1) if n < 1 else (1, n)
    # 当前区间大小比指定的精度大则一直循环
    while right - left > precision:
        mid = (left + right) / 2
        if mid * mid > n:
            right = mid
        else:
            left = mid
    # 最后的答案是当前区间的中点
    return (left + right) / 2

In [ ]:
print(sqrt1(2, 1e-10))

## 方法2：牛顿法

1. **初始估计与切线方程**  
   假设我们要求解 $ f(x)=0 $，先给定一个初始近似值 $ x_0 $。在 $ x=x_0 $ 处，函数值为 $ f(x_0) $，切线的斜率为 $ f'(x_0) $。  
   根据斜截式，给定初始点$(x_0)$，则切线方程为  
    $$
    y - y_0 = f'(x_0)(x - x_0)
    $$
    也就是：
    $$
    y = f(x_0) + f'(x_0)(x - x_0)
    $$

2. **求切线与 $ x $ 轴的交点**  
   切线与 $ x $ 轴的交点满足 $ y=0 $，将 $ y=0 $ 代入上式，有  
   $$
   0 = f(x_0) + f'(x_0)(x - x_0)
   $$

3. **解方程得到新的估计**
   $$
   x = x_0 - \frac{f(x_0)}{f'(x_0)}
   $$
   我们将这个 $ x $ 称为新的估计 $ x_1 $，即  
   $$
   x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}
   $$

这样，我们就得到了牛顿法的递推公式（**重点记忆！！！记这个就行了，这样不管是平方跟还是立方根，现场带入f(x)然后算即可！**）：
$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}
$$


### 应用于求 $\sqrt{n}$ 的问题
令
$$
x = \sqrt{n}
$$
变换
$$
x^2 = n
$$
$$
0 = x^2 - n
$$

也就是求
$$
f(x) = x^2 - n
$$
的零点，则
$$
f'(x) = 2x
$$
根据递推公式，代入 $ f(x) $ 和 $ f'(x) $：
$$
x_{k+1} = x_k - \frac{x_k^2 - n}{2x_k} = \frac{1}{2}\left( x_k + \frac{n}{x_k} \right)
$$
这就是用于计算 $\sqrt{n}$ 的牛顿迭代公式。

直接记忆：
$$
x_1 = \frac{1}{2}\left( x_0 + \frac{n}{x_0} \right)
$$



In [ ]:
def sqrt2(n: float, precision=1e-10) -> float:
    x = 1 if n < 1 else n  # 初始估计x0
    prev_x = 0  # 记录前一个x来记录每次更新的步长
    # 如过更新步长大于指定精度则一直循环
    while abs(x - prev_x) > precision:
        prev_x = x
        x = 0.5 * (x + n / x)  # 递推公式
    return x

# 如果参数直接就是x，那么可以先令n = x，就相当于求根号n了
def sqrt2_(x, precision=1e-10):
    n = x
    x = 1 if n < 1 else n  # 初始估计x0
    prev_x = 0
    while abs(x - prev_x) > precision:
        prev_x = x
        x = 0.5 * (x + n / x)
    return x

In [ ]:
print(sqrt2(2, 1e-10))
print(sqrt2_(2, 1e-10))

### 补充：立方根

还是使用牛顿法通用的迭代公式：
$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}
$$

和平方根一样，要求解的是 $f(x) = x^3 - n = 0$，得到递推公式：
$$
x_{k+1} = x_k - \frac{x_k^3 - n}{3x_k^2}
$$

In [ ]:
def cbrt(n: float, precision=1e-10) -> float:
    x = 1 if n < 1 else n  # 初始猜测
    prev_x = 0
    while abs(x - prev_x) > precision:
        prev_x = x
        x = x - (x**3 - n) / (3 * x**2)  # 迭代公式（未化简）
    return x

# 方法3：梯度下降法

梯度下降主要通过求梯度为0的点，得到凸函数的全局最小值。
首先构建损失函数为凸函数的目标函数, 使得目标函数的最小值对应的是我们要求的值。

由：
$$
x^2 = n
$$

我们可以构建一个损失函数：
$$
f(x)=\tfrac12\,(x^2-n)^2
$$

这里不直接用 $x^2 - n$ 是因为其最小值对应$x$的不是我们想要的$x$。

构建了损失函数以后，就可以用梯度下降法来求全局最优的$x$的值，即为我们要求的 $\sqrt{n}$ 的值。

它在 $x=\pm\sqrt{n}$ 处取到全局最小（值为 0），且 $f'(x)=2x(x^2-n)$。
用梯度下降做迭代：

$$
x_{k+1}=x_k-\eta\;\underbrace{2x_k(x_k^2-n)}_{\nabla f(x_k)}.
$$

早停条件：残差 $|x_k^2-n|$ 很小

**形式**：$|x_k^2-n|\le \varepsilon$。

**含义**：直接约束方程误差 $x^2=n$，是“问题相关”的收敛标准。


In [ ]:
def sqrt_gd_simple(n, lr=1e-4, eps=1e-12, max_iter=10000):
    assert n >= 0, "n必须非负"
    if n == 0:
        return 0.0
    # 简单正初值：n>=1 用 n；n<1 用 1.0
    x = n if n >= 1.0 else 1.0
    for k in range(1, max_iter + 1):
        # 梯度下降
        grad = 2.0 * x * (x * x - n)
        x = x - lr * grad  # 负梯度方向更新
        # 早停条件：方程残差小于阈值时停止更新
        if abs(x * x - n) <= eps:
            return x
    return x